In [34]:
import pandas as pd
import numpy as np

# 1. Предобработка данных для модели

In [35]:
# загрузка сырых данных
train_raw = pd.read_csv('./features.csv', index_col = 'match_id')
test = pd.read_csv('./features_test.csv', index_col = 'match_id')

In [36]:
# для начала необходимо удалить признаки связанные с исходом матча из train_raw
train_raw.drop(['duration', 
         'tower_status_radiant', 
         'tower_status_dire', 
         'barracks_status_radiant', 
         'barracks_status_dire'],
               axis = 1,
               inplace = True)

train_raw = train_raw.fillna(0)

In [37]:
# разделим тренировочный датасет на тренировочные данные и ответы
X = train_raw
y = train_raw['radiant_win'].to_frame()
X.drop(['radiant_win'], axis=1, inplace=True)

# 2. Попробуем градиентный бустинг

In [38]:
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.cross_validation import KFold, cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [39]:
# определим фолды
kf = KFold(y.size, n_folds = 5, shuffle = True, random_state = 100)

In [40]:
gb_scores = []
num_trees = [10, 20, 30, 40, 50, 100, 150]
for n in num_trees:
    print('Trying ', n)
    model = GradientBoostingClassifier(n_estimators=n, random_state=100)
    start_t = datetime.datetime.now()
    scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc', n_jobs=-1)
    print('Time: ', datetime.datetime.now() - start_t)
    print(scores)
    gb_scores.append(np.mean(scores))
    

Trying  10
Time:  0:00:47.719590
[ 0.66165034  0.66480756  0.6675309   0.6610796   0.66778999]
Trying  20
Time:  0:01:30.434678
[ 0.68208814  0.6812496   0.68436094  0.67760577  0.68558318]
Trying  30
Time:  0:01:32.815543
[ 0.68961544  0.68761653  0.6917235   0.68538975  0.69276825]
Trying  40
Time:  0:01:52.265892
[ 0.69439493  0.69207347  0.6968409   0.68971842  0.69796541]
Trying  50
Time:  0:02:12.723152
[ 0.69764     0.69569809  0.69987237  0.69304983  0.70124807]
Trying  100
Time:  0:04:15.133076
[ 0.70670196  0.70617433  0.70911064  0.70218747  0.71042098]
Trying  150
Time:  0:06:18.067154
[ 0.71060788  0.7102596   0.71295173  0.70641361  0.71446841]


In [1]:
# Построим график roc_auc
plt.plot(num_trees, gb_scores)
plt.xlabel('trees')
plt.ylabel('score')
plt.show()

NameError: name 'plt' is not defined

# Выводы
* Кросс-валидация на 30 деревьях заняла 1мин 32сек и дала ROC-AUC score ~ 0.69
<br>
* 30-50 решающих деревьев представляется оптимальным. 
<br>
* Возможно увеличение числа деревьев, что несколько повышает качество до ~0.71-0.72
<br>
* При увеличении числа деревьев, возможно стоит попробовать использовать урезанную выборку для обучения. Возможно имеет смысл ограничить глубину дерева. Эти меры могут привести к уменьшению времени обучения при некотором росте качества модели